In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
line_counter = 0
header = []
nutrition_data = []

with open('nutrition.csv', 'rt', encoding = 'utf-8') as f:
    while 1:
        data = f.readline()
        print(data) # data에 파일을 한 줄씩 불러옴
        if not data: break 
        if line_counter == 0:
            header = data.split(",") # 맨 첫 줄은 header로 저장
        else:
            nutrition_data.append(data.split(","))
        line_counter += 1

﻿Food groups,Food Code,Food and Description,Food and Description,Energy,Water,Protein,Fat,Ash,Carbohydrate,Total sugars,Sucrose,Glucose,Fructose,Lactose,Maltose,Galactose,Total Dietary Fiber,Water Soluble Dietary Fiber,Water Insoluble Dietary Fiber,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Manganese,Selenium,Molybdenum,Iodine,Retinol,β-Carotene,Thiamin,Total Riboflavin,Total Niacin,Niacin equivalents,Nicotinic acid,Nicotinamide,Pantothenic acid,Total Vitamin B6,Pyridoxine,Biotin,Dietary Folate Equivalent,Food Folate,Folic acid,Cyanocobalamin,Total Ascorbic Acid,Vitamin D (Ergocalciferol + Cholecalciferol),Ergocalciferol,Cholecalciferol,Total Vitamin E,α-tocopherol,β-tocopherol,γ-tocopherol,δ-tocopherol,α-tocotrienol,β-tocotrienol,γ-tocotrienol,δ-tocotrienol,Total Vitamin K,Phylloquinone,Parquinone,Sum of 19 amino acids,Total essential amino acid,Isoleucine,Leucine,Lysine,Methionine,Phenylalanine,Threonine,Tryptophan,Valine,Histidine,Arginine,Total non-essential ami

채소류,F087000B120a,"배추, 싹, 생것","Chinese cabbage, Sprouts, Raw",63,86.3,3.4,4.3,0.7,5.3,-,-,-,-,-,-,-,4.3,0.6,3.7,61,1.1,-,150,123,8,-,-,-,-,-,-,0,817,0.65,0.28,1,-,-,-,-,-,-,-,-,-,-,-,17,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,농진청('11)

채소류,F0870010000a,"배추, 노랑봄배추(노지), 생것","Chinese cabbage, Norangbombaechu, Outdoor, raw",13,95.5,1,-,0.5,3,-,-,-,-,-,-,-,-,-,-,24,0.5,-,22,108,9,-,-,-,-,-,-,0,22,0.05,0.04,0.2,-,-,-,-,-,-,-,-,-,-,-,11,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,-,농진청('08)

채소류,F0870010009c,"배추, 노랑봄배추(노지), 데친것","Chinese cabbage, Norangbombaechu, Outdoor, Blanched",17,94.6,1.2,-,0.3,3.9,-,-,-,-,-,-,-,-,-,-,39,1,-,25,94,12,-,-,-,-,-,-,0,6,0.1,0.01,0.3,-,-,-,-,-,-,-,-,-,-,-,6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,

육류,I027006D910a,"소고기, 한우(1등급), 사태(뭉치사태), 생것","Beef meat, Hanwoo, 1 grade, Shank(Heel meat), Raw",146,71.2,24.04,4.86,1.02,0,0,0,0,0,0,0,0,0,0,0,5,2.84,23,204,338,57,5.61,0.058,0.014,21.7,1.02,0,1,0,0.06,0.187,1.911,-,-,-,1.56,-,0.046,2.38,4,4,-,1.51,0.7,0,0,0,0.26,0.18,0,0.03,0,0.03,0,0.02,0,-,0,-,22908,11218,1144,1655,2075,519,923,1214,227,1112,891,1458,11690,804,245,1304,2285,3734,1240,1058,1020,0,56.89,4.64,0.17,1.86,0,0,0,2.5,4.82,0,134.8,12.1,1145.42,46.18,441.05,2.47,10.91,0.74,57.04,0.9,2.49,36.99,240.53,0,2063.48,131.47,14.7,0.22,0,0.2,162.37,4.47,0,3.16,0,25.12,0.67,0,0.2,4,0,0.01,0.19,0.09,55.1,23.26,12.17,0.1,-,농진청('16)

육류,I027006D920a,"소고기, 한우(1등급), 사태(상박살), 생것","Beef meat, Hanwoo, 1 grade, Shank(Conical meat), Raw",165,68.7,21.38,8.17,0.84,0,0,0,0,0,0,0,0,0,0,0,4,2.56,18,176,294,59,6.24,0.054,0.014,22.12,0.62,0.33,3,0,0.076,0.189,2.287,-,-,-,0.324,-,0.044,2.32,4,4,-,2.69,0.94,0,0,0,0.35,0.26,0.01,0.04,0,0.03,0,0.02,0,-,0,-,19823,9449,902,1702,1718,427,728,798,210,963,655,

우유및유제품류,M0080012319a,"요구르트, 액상, 저당","Yoghurt, Liquid type, Low-sugar",45,88.2,1.11,0.02,0.29,10.38,7.16,0,1.34,3.06,2.2,0,0,0.7,0.4,0.3,45,0.1,4,34,60,18,0.11,0,0.003,1.04,1.26,0,0,0,0,0.081,0.037,-,-,-,0,-,0,0,14,14,-,0,0.46,0,0,0,0,0,0,0,0,0,0,0,0,-,0,-,1035,487,48,98,65,24,69,52,9,67,27,28,548,43,0,38,92,219,22,79,55,0,0.65,0.02,0,0.02,0,0,0,0,0,0,2.24,0,9.68,0,4.03,0,0,0,0,0,0,0,0,0,4.8,0,0,0,0,0,0.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,농진청('15)

우유및유제품류,M0080011389a,"요구르트, 액상(농후), 딸기맛","Yoghurt, Thick fermented milk, Liquid type, Strawberry flavored milk",85,80.2,3.29,2.22,0.8,13.49,10.51,0,2.51,4.65,0,0,3.35,0.2,0,0.2,133,0.08,12,92,176,46,0.4,0.001,0.009,3.52,2.49,31.61,23,0,0.009,0.42,0.288,-,-,-,0.328,-,0,3.34,15,15,-,-,0.91,0,0,0,0.06,0.04,0,0.02,0,0,0,0,0,-,0,-,3214,1478,146,335,249,72,147,138,33,169,81,108,1736,149,40,106,248,634,61,327,171,-,12,2.11,0.06,1.44,55.01,37.68,21.17,48.42,79.16,1.77,231.68,18.79,641.49,11.9,284.53,4.44,0.73,2.19,0,1.31,0.5,16.29,36.83,0,436.73,1

In [3]:
nutrition_data = pd.read_csv('./nutrition.csv', encoding = 'utf-8')
food_ratings_data = pd.read_csv('./food_ratings.csv', encoding = 'utf-8')

In [4]:
nutrition_data

,Food groups,Food Code,Food and Description,Food and Description.1,Energy,Water,Protein,Fat,Ash,Carbohydrate,...,Docosahexaenoic (DHA),Total n-3 polyunsaturated Fatty acid,Total n-6 polyunsaturated Fatty acid,Trans fatty acid,Trans oleic,Trans linolenic,Trans linolenic.1,Sodium Chloride Equivalent,Refuse,Source of Data
0,곡류및그제품,A001001A010a,"귀리, 겉귀리, 도정, 생것","Oat(Covered oat), Polished, Raw",373,9.4,11.4,3.7,2,73.5,...,-,-,-,-,-,-,-,0,0,농진청('93)
1,곡류및그제품,A001002A010a,"귀리, 쌀귀리, 도정, 생것","Oat(Naked oat), Polished, Raw",371,9.7,14.3,3.8,1.8,70.4,...,-,-,-,-,-,-,-,0,0,농진청('93)
2,곡류및그제품,A0020000009a,"귀리, 오트밀","Oat, Oatmeal",382,12,13.2,8.2,1.7,64.9,...,-,-,-,-,-,-,-,0,0,농진청('11)
3,곡류및그제품,A003000A010a,"기장, 도정, 생것","Prosomillet, Polished, Raw",360,11.3,11.2,1.9,1,74.6,...,-,-,-,-,-,-,-,0,0,농진청('06)
4,곡류및그제품,A003001A010a,"기장, 찰기장, 도정, 생것","Prosomillet, Glutinous, Polished, Raw",365,11.3,12.46,3.09,1.24,71.91,...,0,0.04,1.91,0.01,0,3.12,3.53,0,0,농진청('13)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083,기타,T0250010009a,"팽창제, 베이킹파우더","Leavening agent, Baking powder, Straight phosp...",51,4,0.1,0,71.8,24.1,...,0,-,-,-,-,-,-,20,-,USDA('15)
3084,기타,T0250030001a,"팽창제, 효모, 말린것","Leavening agent, Yeast, Dried",313,8.7,37.1,6.8,4.3,43.1,...,0,0.01,0.03,-,-,-,-,0.3,0,JAPAN('15)
3085,기타,T0260000009a,프로폴리스,Propolis,642,2.1,0.5,50.3,0.2,46.9,...,-,-,-,-,-,-,-,0,-,농진청('05)
3086,기타,T0290002601a,"흰점박이 꽃무지, 유충, 말린것","Protaetia brevitarsis, Larva, Dried",463,1.6,54.13,18.52,5.76,19.99,...,0,0.06,1.52,0.14,27.74,39.71,72.04,0.5,0,농진청('15)


In [5]:
food_ratings_data

,userId,Food Code,rating,timestamp
0,1,A001001A010a,2.5,1260759144
1,1,A001002A010a,3.0,1260759179
2,1,A0020000009a,3.0,1260759182
3,1,A003000A010a,2.0,1260759185
4,1,A003001A010a,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [6]:
#전처리
food_ratings_data.drop('timestamp', axis = 1, inplace = True)
food_ratings_data.head()

,userId,Food Code,rating
0,1,A001001A010a,2.5
1,1,A001002A010a,3.0
2,1,A0020000009a,3.0
3,1,A003000A010a,2.0
4,1,A003001A010a,4.0


In [8]:
user_food_data = pd.merge(food_ratings_data, nutrition_data, on = 'Food Code')
user_food_data.head()

,userId,Food Code,rating,Food groups,Food and Description,Food and Description.1,Energy,Water,Protein,Fat,...,Docosahexaenoic (DHA),Total n-3 polyunsaturated Fatty acid,Total n-6 polyunsaturated Fatty acid,Trans fatty acid,Trans oleic,Trans linolenic,Trans linolenic.1,Sodium Chloride Equivalent,Refuse,Source of Data
0,1,A001001A010a,2.5,곡류및그제품,"귀리, 겉귀리, 도정, 생것","Oat(Covered oat), Polished, Raw",373,9.4,11.4,3.7,...,-,-,-,-,-,-,-,0,0,농진청('93)
1,1,A001002A010a,3.0,곡류및그제품,"귀리, 쌀귀리, 도정, 생것","Oat(Naked oat), Polished, Raw",371,9.7,14.3,3.8,...,-,-,-,-,-,-,-,0,0,농진청('93)
2,1,A0020000009a,3.0,곡류및그제품,"귀리, 오트밀","Oat, Oatmeal",382,12,13.2,8.2,...,-,-,-,-,-,-,-,0,0,농진청('11)
3,1,A003000A010a,2.0,곡류및그제품,"기장, 도정, 생것","Prosomillet, Polished, Raw",360,11.3,11.2,1.9,...,-,-,-,-,-,-,-,0,0,농진청('06)
4,1,A003001A010a,4.0,곡류및그제품,"기장, 찰기장, 도정, 생것","Prosomillet, Glutinous, Polished, Raw",365,11.3,12.46,3.09,...,0,0.04,1.91,0.01,0,3.12,3.53,0,0,농진청('13)


In [49]:
user_food_data.shape

(81, 138)

In [50]:
user_food_data

array([[1, 'A001001A010a', 2.5, ..., '0', '0', "농진청('93)"],
       [1, 'A001002A010a', 3.0, ..., '0', '0', "농진청('93)"],
       [1, 'A0020000009a', 3.0, ..., '0', '0', "농진청('11)"],
       ...,
       [2, 'S0800000007a', 4.0, ..., '1.2', '0', "JAPAN('15)"],
       [2, 'S0800020009a', 4.0, ..., '1', '0', "농진청('16)"],
       [2, 'S0840000009d', 4.0, ..., '0.4', 0, "식약('11)"]], dtype=object)

In [53]:
df = pd.DataFrame(user_food_data)
df.loc[:].replace('-', np.nan, inplace=True)
df = df.fillna(0)

In [54]:
df

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,1,A001001A010a,2.5,곡류및그제품,"귀리, 겉귀리, 도정, 생것","Oat(Covered oat), Polished, Raw",373,9.4,11.4,3.7,...,0,0,0,0,0,0,0,0,0,농진청('93)
1,1,A001002A010a,3.0,곡류및그제품,"귀리, 쌀귀리, 도정, 생것","Oat(Naked oat), Polished, Raw",371,9.7,14.3,3.8,...,0,0,0,0,0,0,0,0,0,농진청('93)
2,1,A0020000009a,3.0,곡류및그제품,"귀리, 오트밀","Oat, Oatmeal",382,12,13.2,8.2,...,0,0,0,0,0,0,0,0,0,농진청('11)
3,1,A003000A010a,2.0,곡류및그제품,"기장, 도정, 생것","Prosomillet, Polished, Raw",360,11.3,11.2,1.9,...,0,0,0,0,0,0,0,0,0,농진청('06)
4,1,A003001A010a,4.0,곡류및그제품,"기장, 찰기장, 도정, 생것","Prosomillet, Glutinous, Polished, Raw",365,11.3,12.46,3.09,...,0,0.04,1.91,0.01,0,3.12,3.53,0,0,농진청('13)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2,S0780000007a,4.0,조리가공식품류,"핫도그, 냉동","Corndog, Frozen",267,44.5,8.09,10.55,...,7.66,0.13,1.51,0.08,32.83,24.72,20.77,1.2,0,농진청('16)
77,2,S0800000009a,4.0,조리가공식품류,"햄버거, 소고기패티","Hamburger, Beef patty",271,41.3,11.94,8.94,...,0,0.06,0.66,0.33,265.49,56.07,13.1,1.5,0,농진청('16)
78,2,S0800000007a,4.0,조리가공식품류,"햄버거 스테이크, 냉동","Hamburger steak, Frozen",223,59.2,13.3,13.4,...,0,0,0,0,0,0,0,1.2,0,JAPAN('15)
79,2,S0800020009a,4.0,조리가공식품류,"햄버거, 소고기패티, 토마토, 양상추, 양파","Hamburger, Beef patty, Tomato, Lettuce, Onion",229,57.8,9.55,13.23,...,0,0.39,3.15,0.19,118.17,50.49,20,1,0,농진청('16)


In [55]:
user_food_data = df.values
user_food_data.shape

(81, 138)

In [56]:
type(user_food_data)

numpy.ndarray

In [57]:
user_food_data

array([[1, 'A001001A010a', 2.5, ..., '0', '0', "농진청('93)"],
       [1, 'A001002A010a', 3.0, ..., '0', '0', "농진청('93)"],
       [1, 'A0020000009a', 3.0, ..., '0', '0', "농진청('11)"],
       ...,
       [2, 'S0800000007a', 4.0, ..., '1.2', '0', "JAPAN('15)"],
       [2, 'S0800020009a', 4.0, ..., '1', '0', "농진청('16)"],
       [2, 'S0840000009d', 4.0, ..., '0.4', 0, "식약('11)"]], dtype=object)

In [62]:
ufd = np.delete(user_food_data, 1, axis = 1)

In [64]:
ufd = np.delete(ufd, 2, axis = 1)

In [67]:
ufd = np.delete(ufd, 2, axis = 1)

In [69]:
ufd = np.delete(ufd, 2, axis = 1)

In [72]:
ufd = np.delete(ufd, 133, axis = 1)

In [73]:
ufd

array([[1, 2.5, '373', ..., 0, '0', '0'],
       [1, 3.0, '371', ..., 0, '0', '0'],
       [1, 3.0, '382', ..., 0, '0', '0'],
       ...,
       [2, 4.0, '223', ..., 0, '1.2', '0'],
       [2, 4.0, '229', ..., '20', '1', '0'],
       [2, 4.0, '74', ..., 0, '0.4', 0]], dtype=object)

In [74]:
ufd.shape

(81, 133)

In [77]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(ufd)
matrix.shape

ValueError: could not convert string to float: 'tr'

In [37]:
matrix[0]

NameError: name 'matrix' is not defined

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

In [ ]:
corr2 = corr[:200, :200]
corr2.shape

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(corr2)

In [ ]:
movie_title = user_movie_rating.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)")

In [ ]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]